#Setup

In [ ]:
from google.colab import drive
drive.mount('/content/drive',  force_remount=True)
%cd '/content/drive/Othercomputers/Lab Desktop 2/Desktop/Research_EMS'

In [ ]:
%pip install geopandas
%pip install rasterio
%pip install momepy
%pip install contextily
%pip install ray
%pip install darts
%pip install folium
%pip install mapclassify
%pip install xyzservices

In [ ]:
from preprocess import *
from mpl_toolkits.axes_grid1 import make_axes_locatable
import xyzservices.providers as xyz
from collections import Counter
import plotly.figure_factory as ff

# Data preprocessing

## Preprocess large data

In [ ]:
# preprocess and save ambulance data
# data = rawData('./data/ambulance/virginiaBeach_ambulance_timeData.csv')
# data = addOrigin(data, './data/rescueTeamLocation/rescueStations.txt')
# data = geoCoding(data.loc['2013-01-01' : '2013-12-31', :], './data/ambulance/geocoded_saved/20130101-20131231.csv')

In [ ]:
# roads = readRoads('./data/roads/Streets.shp')
# roads = makeSurface4Lines(roads, './data/roads/Road_Surfaces.shp', scale = 2.7)
# roads = getWaterDepthOnRoads(roads, './data/inundation/tifData/depth_objID_35.tif', './data/inundation/croppedByRoads/croppedByRoads.tif')

# roads.drop(['line', 'midpoint','buffers','buffersUnscaled'], axis = 1).to_file('./data/roads/savedInundatedRoads/roads_with_objID_35.shp')
# roads['line'].to_file('./data/roads/savedInundatedRoads/roads_with_objID_35_line.shp')
# roads['midpoint'].to_file('./data/roads/savedInundatedRoads/roads_with_objID_35_midpoint.shp')
# roads['buffers'].to_file('./data/roads/savedInundatedRoads/roads_with_objID_35_buffers.shp')
# roads['buffersUnscaled'].to_file('./data/roads/savedInundatedRoads/roads_with_objID_35_buffersUnscaled.shp')

# consider bridges in road network (PENDING)
# bridges = gpd.read_file('./data/roads/bridges/bridgePolygon.shp').to_crs(str(inundation.crs))
# inundation_cropped = inundationCutter(inundation, bridges, True, True)

## Reload and additional preprocessing

In [ ]:
# reload data and build geoDataFrame
data = reLoadData('./data/ambulance/geocoded_saved/20160101-20161015.csv')
roads = reLoadRoads('./data/roads/savedInundatedRoads/roads_with_objID_35.shp')

In [ ]:
# create graph
graph = roads2Graph(roads)
# showGraphRoads(roads, graph)
# NOTE: the graph is un-directed right now, the logic should be checked if changed to directed

# read the location of rescue squads and attach them to nodes
rescue = readRescue('./data/rescueTeamLocation/rescueStations.txt', 'EPSG:4326', roads)

In [ ]:
# additional info for descriptive analysis
# assign records to graph edges
data = assignGraphEdge(data, roads, 'RescueSquadPoint', 'OriginRoadID', 'Origin2RoadDist')
data = assignGraphEdge(data, roads, 'IncidentPoint', 'DestinationID', 'Destination2RoadDist')

# find nearest rescue station
# data = nearestRescueStation(data, rescue)

# find the top nearest rescue stations
data = nearnessObediance(data, rescue, graph)

# calculate shortest path length and ave speed
data = assumedAveSpeed(data, rescue, graph)

# Disruption analysis

In [ ]:
def _addPathLen2Graph(graph, rescue, weight, newAttribute_rescueSquad, newAttribute_pathLen, newAttribute_pathList = None):
    # some roads are disconnected from all the rescue station even in normal time (as the raw data indicates)
    voronoi = nx.voronoi_cells(graph, set(rescue.OBJECTID_nearestRoad.unique()), weight = weight)
    for rescueSquad, destinations in zip(voronoi.keys(), voronoi.values()):
        if rescueSquad == 'unreachable':
            print(len(destinations), 'nodes are unreachable when building voronoi for', newAttribute_pathLen)
            for des in destinations:
                graph.nodes[des][newAttribute_rescueSquad] = np.nan
                graph.nodes[des][newAttribute_pathLen] = math.inf # set path len to inf if it's disconnected from rescues
#                 print('NOTE: node', des, 'is unreachable when building voronoi for', newAttribute_pathLen)
        else:
            for des in destinations:
                shortestPathLen = nx.shortest_path_length(graph, source = rescueSquad, target = des, weight = weight)
                graph.nodes[des][newAttribute_pathLen] = shortestPathLen
                graph.nodes[des][newAttribute_rescueSquad] = rescueSquad
                if newAttribute_pathList:
                    shortestPath = nx.shortest_path(graph, source = rescueSquad, target = des, weight = weight)
                    graph.nodes[des][newAttribute_pathList] = shortestPath
                if shortestPathLen == 0:
                    graph.nodes[des][newAttribute_pathLen] = 1
                if shortestPathLen == math.inf:
                    graph.nodes[des][newAttribute_rescueSquad] = math.inf
    return graph, voronoi

def _addDisruption(graph, roads, newAttribute = 'weightWithDisruption', threshold = 3):
    nx.set_edge_attributes(graph, nx.get_edge_attributes(graph, "weight"), newAttribute)
    disruptedRoads = roads[roads['waterDepth'] >= threshold]['OBJECTID'].to_list()
    for disruption in disruptedRoads:
        for edge in graph.edges(disruption):
            graph.edges()[edge][newAttribute] = math.inf # set edge weight to inf if it's disrupted by inundation
    return graph

def _changeValue4DisruptedRoad(roads, graph, threshold = 3):
    # the disrupted road itself is not disconnected, so assign the shortestPath of adjancent road to this road
    for disruption in roads[roads['waterDepth'] >= threshold]['OBJECTID'].to_list():
        pathLen = []
        edgeNum = []
        for edge in graph.edges(disruption):
            pathLen.append(graph.nodes()[edge[1]]['shortestPathLenWithDisruption'])
            edgeNum.append(edge[1])
        if pathLen != []: # in case there are disconnected single node
            graph.nodes()[disruption]['shortestPathLenWithDisruption'] = min(pathLen)
            if min(pathLen) != math.inf:
                graph.nodes()[disruption]['rescueAssignedWithDisruption'] = edgeNum[pathLen.index(min(pathLen))]
            else:
                graph.nodes()[disruption]['rescueAssignedWithDisruption'] = np.nan
    return graph

def runRoutingWithDisruption(graph, rescue, roads):
    graph, _ = _addPathLen2Graph(graph, rescue, 'weight', 'rescueAssigned', 'shortestPathLen', 'shortestPathList')
    graphDisrupted = _addDisruption(graph, roads, threshold = 1)
    graph, _ = _addPathLen2Graph(graphDisrupted, rescue, 'weightWithDisruption', 'rescueAssignedWithDisruption', 'shortestPathLenWithDisruption', 'shortestPathListWithDisruption')
    graph = _changeValue4DisruptedRoad(roads, graph, threshold = 1)
    return graph

def getDisruptionRatio(graph):
    nx.set_node_attributes(graph,
                           {x[0]: y[1]/x[1] if y[1]/x[1] != math.inf else np.nan \
                            for x, y in zip(nx.get_node_attributes(graph, "shortestPathLen").items(),
                                            nx.get_node_attributes(graph, "shortestPathLenWithDisruption").items() ) },
                           'travelTimeIncreaseRatio')
    roads['travelTimeIncreaseRatio'] = roads['OBJECTID'].map(nx.get_node_attributes(graph, "travelTimeIncreaseRatio"))
    return graph

def removeDisconnectedNodes(graph):
    largest_cc = max(nx.connected_components(graph), key = len)
    nodes_to_remove = [node for node in graph.nodes if node not in largest_cc]
    graph.remove_nodes_from(nodes_to_remove)
    return graph

# calculate ratios
graph = runRoutingWithDisruption(graph, rescue, roads)
graph = getDisruptionRatio(graph)
graph = removeDisconnectedNodes(graph)

In [ ]:
roads_fromGraph = pd.DataFrame.from_dict(dict(graph.nodes(data = True)), orient = 'index')
roads_fromGraph = gpd.GeoDataFrame(roads_fromGraph, geometry = 'line').set_crs('EPSG:3857').to_crs('EPSG:4326')

# Figs and Charts

In [ ]:
def showRoadsWithValues(roads, values, vmax = 6, vmin = 0, saveAddr = None, figsize = (10, 10), cmap = 'OrRd'):
    roads = roads.set_geometry('line')
    roads = roads.to_crs('EPSG:4326')
    samplePoints_lat = roads.apply(lambda row: row.line.coords[0][1], axis = 1)
    samplePoints_lon = roads.apply(lambda row: row.line.coords[0][0], axis = 1)
    roads = roads.loc[samplePoints_lat > 36.72]
    roads = roads.loc[samplePoints_lon < -75.98]
    roads = roads.loc[samplePoints_lon > -76.16]

    # roads = roads.set_geometry('midpoint') # open it if draw points

    fig, ax = plt.subplots(figsize = figsize)

    cax = make_axes_locatable(ax).append_axes("right", size = "2%", pad = 0.1,)
    cax.tick_params(labelsize = 25)

    roads.plot(
        ax = ax,
        cax = cax,
        column = values,
        zorder = 5,
        cmap = cmap,
        # color = 'deeppink',
        legend = True,
        vmax = vmax,
        vmin = vmin,
        # linewidth = roads[values], # open if adjust line width
        linewidth = 3,
        # marker = 'o', # open it if draw points
        # markersize = 600, # open it if draw points
        )

    cx.add_basemap(ax, crs = roads.crs, source = cx.providers.CartoDB.Positron, zoom = 13)
    cx.add_basemap(ax, crs = roads.crs, source = cx.providers.Stamen.TonerLabels, zoom = 13, alpha = 0.6)

    ax.set_axis_off()
    if saveAddr:
        fig.savefig(saveAddr, dpi = 300)
        plt.close(fig)
    else:
        plt.show(ax)


def incidentMap(data, timeSelectStart, timeSelectEnd, sizeMax, ifLog):
    # general spatial dist of incidents
    if ifLog == True:
        data['LogResponseTime'] = np.log2(data['ResponseTime'])
        colorData = "LogResponseTime"
        range_color = [0, 20]
    elif ifLog == False:
        colorData = "ResponseTime"
        range_color = [0, 45000]
    px.set_mapbox_access_token(open("mapboxToken.txt").read())
    dataSelected = data.loc[timeSelectStart: timeSelectEnd, :].dropna()
    fig = px.scatter_mapbox(lat = dataSelected.IncidentPoint.y, lon = dataSelected.IncidentPoint.x, color = dataSelected.ResponseTime,
                            color_continuous_scale = px.colors.sequential.Sunsetdark, range_color = range_color,
                            size = dataSelected.ResponseTime,
                            size_max = sizeMax,
                            zoom = 9.5, width = 750, height = 500)
    return fig

def responsTimeScatter(data):
    # show the surge of the response time
    fig = go.Figure(data = go.Scatter(x = data.index, y = data['ResponseTime'], mode='markers', marker_color = data['ResponseTime'],))
    fig.update_layout(xaxis_title = "Datatime", yaxis_title = "Response time (s)",)
    return fig

def processingTimeProportionDist(data):
    df = pd.DataFrame((data.EnRouteTime / data.ResponseTime))
    df = df.rename(columns = {0: "Proportion of Preparation Time"})
    fig = px.histogram(df, x = "Proportion of Preparation Time",
                        nbins = 75, template = 'seaborn', histnorm = 'probability', opacity = 0.75,
                        width = 700, height = 500)
    fig.update_layout(yaxis_title = 'Probability')
    return fig

def proximityOrderDist(data):
    df = data.copy()
    df['Flooding'] = 'Normal'
    df.loc['2016-10-08 11:59:59': '2016-10-09 23:59:59', ['Flooding']] = 'Flooding'
    fig = px.histogram(df[df.NearestOrder < 10],
                 x = "NearestOrder",
    #              color = "Flooding",
                 template = 'seaborn',
                 histnorm = 'probability',
                 barmode = "overlay",
                 opacity = 0.75,
                 width = 700, height = 500,
                )
    fig.update_layout(yaxis_title = 'Probability', xaxis_title = 'Proximity Order of Origins')
    return fig

def distanceIncreaseRatioDist(data):
    fig = px.histogram(data[(data.DisobediancePathIncrease > 1)],
                 x = "DisobediancePathIncrease",
                 barmode = "overlay",
                 template = 'seaborn', histnorm = 'probability',
                 opacity = 0.75,
                 width = 700, height = 500,)
    fig.update_traces(xbins = dict(start = 1, end = 2, size = 0.1))
    fig.update_layout(yaxis_title = 'Probability', xaxis_title = 'Travel Distance Increase Percentage')
    return fig

def responseTimeWithCallPriorityDist(data):
    fig = px.histogram(data[data.ResponseTime < 5000],
                 x = "ResponseTime",
                 color = "CallPriority",
                 barmode = "overlay",
                 template = 'seaborn',
                 histnorm = 'probability',
                 opacity = 0.75,
                 width = 700, height = 500,
                 nbins = 200,
                )
    fig.update_layout(yaxis_title = 'Probability', xaxis_title = 'Response Time')
    return fig

def averageSpeedPercentStd(data):
    # np.histogram(data.groupby(['OriginRoadID', 'DestinationID']).count().AverageSpeed.values, 10, range = (5, 100)) # keep about 25% of OD when set freqencey above 5
    dataSelect = data.loc[:, ['OriginRoadID', 'DestinationID', 'AverageSpeed']]
    groupByODCount = dataSelect.groupby(['OriginRoadID', 'DestinationID']).count() # any columns indicates count
    groupByODSpeed = dataSelect.groupby(['OriginRoadID', 'DestinationID']).mean()[groupByODCount.AverageSpeed >= 5].rename(columns = {'AverageSpeed': 'AverageSpeed_mean'})
    groupByODSpeed['AverageSpeed_std'] = dataSelect.groupby(['OriginRoadID', 'DestinationID']).std()[groupByODCount.AverageSpeed >= 5]
    groupByODSpeed['AverageSpeed_stdPercent'] = groupByODSpeed['AverageSpeed_std'] / groupByODSpeed['AverageSpeed_mean']
    df = groupByODSpeed['AverageSpeed_stdPercent'].reset_index()
    OriginNum = df.OriginRoadID
    for origin, num in zip(pd.unique(df.OriginRoadID), range(1, pd.unique(df.OriginRoadID).shape[0] + 1)):
        OriginNum = OriginNum.replace(origin, num)
    df['OriginNum'] = OriginNum
    fig = px.box(df, x = 'OriginNum', y = "AverageSpeed_stdPercent", template = 'seaborn', width = 500, height = 750, range_y = (0, 6.5), points = 'suspectedoutliers')
    fig.update_layout(yaxis_title = 'Average Speed Percentage Standard Deviation', xaxis_title = 'Rescue Squad Number')
    return fig

def showAveTime(time, by):
    dataGroupByHour = data.loc[:, [time, by]].groupby(by).mean()
    fig = px.bar(dataGroupByHour.reset_index(), y = time, x = by, text_auto='.3s',)
    return fig

In [ ]:
# # incident time features
# responsTimeScatter(data)
# responseTimeWithCallPriorityDist(data)
# processingTimeProportionDist(data)
# showAveTime('ResponseTime', 'HourInDay')
# showAveTime('ResponseTime', 'DayOfWeek')

# # incident spatial features
# incidentMap(data, '2016-10-08', '2016-10-09', 16, False)
# proximityOrderDist(data)
# distanceIncreaseRatioDist(data)

# incident speed features
averageSpeedPercentStd(data)

In [ ]:
# distance to nearest rescue station in normal time
roads_fromGraph_cor = roads_fromGraph.apply(lambda row: row.line.coords[0][1], axis = 1)
roads_fromGraph_north = roads_fromGraph['shortestPathLen'].loc[roads_fromGraph_cor > 36.72]
showRoadsWithValues(roads_fromGraph, 'shortestPathLen',
                    vmax = roads_fromGraph_north.max(),
                    vmin = 0,
                    saveAddr = './fig/normal_toRescue',
                    figsize = (30,30))

In [ ]:
# average response time in normal time map
data_0 = data[data.EntryDateTime < '2016-10-01 00:00:00'][['ResponseTime', 'DestinationID']].groupby('DestinationID').mean()
roads_0 = roads.merge(data_0, left_on = 'OBJECTID', right_index = True, how = 'left')
roads_0 = roads_0[~roads_0.ResponseTime.isna()]
roads_0 = roads_0.sort_values(by = ['ResponseTime'])
roads_0 = roads_0.iloc[: math.floor(len(roads_0) * 0.90)]
roads_0 = roads_0[roads_0.ResponseTime != 0]
roads_0
showRoadsWithValues(roads_0, 'ResponseTime',
                    vmax = 759,
                    vmin = 99,
                    saveAddr = './fig/normal_responseTimeMap',
                    figsize = (30,30))

In [ ]:
# inundation depth map
showRoadsWithValues(roads, 'waterDepth', vmax = 6, vmin = 0, saveAddr = './fig/inundation', figsize = (30,30), cmap = 'Blues')

In [ ]:
# show road being cut off
showRoadsWithValues(roads[roads.waterDepth > 1], 'waterDepth', vmax = 13, vmin = 1, saveAddr = './fig/inundation_cutOff', figsize = (30,30))

In [ ]:
# get all route being cut off
roadsCutOff = roads[roads.waterDepth > 1].OBJECTID.values
routesCutOff = []
for nodeName in list(graph.nodes()):
    route = graph.nodes[nodeName]['shortestPathList']
    if any(cutOff in route for cutOff in roadsCutOff):
        routesCutOff.append(route)
# count occurance for nodes
nodesCount = dict(Counter([node for route in routesCutOff for node in route]))
# add them to df
roads_0 = roads.copy()
roads_0['cutOffCount'] = np.log2(roads_0['OBJECTID'].map(nodesCount))
# draw map using the new df
showRoadsWithValues(roads_0, 'cutOffCount', vmax = roads_0.cutOffCount.max(), vmin = 0, saveAddr = './fig/routes_cutOff', figsize = (30,30), cmap = 'Oranges')

In [ ]:
# get the affected destination of the EMS routes
roads_0 = roads_0.set_geometry('midpoint')
desCount = dict(Counter([route[-1] for route in routesCutOff]))
roads_0['ifAffected'] = roads_0['OBJECTID'].map(desCount)
roads_0_select = roads_0[roads_0['ifAffected'] == 1]
showRoadsWithValues(roads_0_select, 'ifAffected', vmax = 1, vmin = 0, saveAddr = './fig/routes_destinations', figsize = (30,30), cmap = 'Oranges')

In [ ]:
# micro scale impact and route change
# roads[(roads.travelTimeIncreaseRatio > 2) & (roads.travelTimeIncreaseRatio < 4)] # select des using the roads df
originalRoute = graph.nodes[166]['shortestPathList']
updatedRoute = graph.nodes[166]['shortestPathListWithDisruption']
showRoadsWithValues(roads[roads.OBJECTID.isin(originalRoute)], '', saveAddr = './fig/routes_micro_original', figsize = (30,30))
showRoadsWithValues(roads[roads.OBJECTID.isin(updatedRoute)], '', saveAddr = './fig/routes_micro_updated', figsize = (30,30))

In [ ]:
# travel distance increase ratio estimation
roads_0 = roads.copy()
roads_0['travelTimeIncreaseRatio'] = np.log10(roads_0['travelTimeIncreaseRatio'])
showRoadsWithValues(roads_0, 'travelTimeIncreaseRatio',
                    vmax = 1,
                    vmin = 0,
                    saveAddr = './fig/response_increaseRatio',
                    figsize = (30,30))

In [ ]:
# show the real increase ratio
ratios = roads_fromGraph.travelTimeIncreaseRatio.values
ratios = ratios[~np.isnan(ratios)]
fig = ff.create_distplot([ratios], ['Increase Ratio'], show_hist = False, show_curve = False)
fig.update_layout(font_size = 30)

In [ ]:
# show the distance dist before and after inundation
fig = ff.create_distplot([roads_fromGraph.shortestPathLen,
                          roads_fromGraph.shortestPathLenWithDisruption[roads_fromGraph.shortestPathLenWithDisruption != np.inf]
                          ], ['Before Inundation', 'After Inundation'],
                         show_rug = False,
                         bin_size = 250,
                         )
fig.update_layout(
    width = 1000,
    height = 500,
    font_size = 15
    )
fig.show()

In [ ]:
# draw the real response time inrease ratio map

# fill the nan in dataDuringFlooding
dataDuringFlooding = data.loc[(data.index > '2016-10-08 18:00:00') & (data.index < '2016-10-9 18:00:00')]
dataDuringFlooding_na = dataDuringFlooding[dataDuringFlooding.ResponseTime.isna()]
dataDuringFlooding_na = dataDuringFlooding_na.reset_index()
dataDuringFlooding_na.CloseDateTime = pd.to_datetime(dataDuringFlooding_na.CloseDateTime)
dataDuringFlooding_na.ResponseTime = dataDuringFlooding_na.CloseDateTime - dataDuringFlooding_na.CallDateTime
dataDuringFlooding_na['ResponseTime'] = dataDuringFlooding_na.ResponseTime.dt.total_seconds()
dataDuringFlooding_na = dataDuringFlooding_na.set_index('CallDateTime')
dataDuringFlooding.update(dataDuringFlooding_na['ResponseTime'])
# get the response time during flooding in real world dataset
responseDuringFlooding = dataDuringFlooding[['DestinationID', 'ResponseTime']].groupby('DestinationID').mean()
# get response time during normal time in real world dataset
dataDuringNormal = data.loc[(data.index < '2016-10-08 18:00:00') | (data.index > '2016-10-9 18:00:00')]
responseDuringNormal = dataDuringNormal[dataDuringNormal.DestinationID.isin(responseDuringFlooding.index.tolist())][['DestinationID', 'ResponseTime']].groupby('DestinationID').mean()

# calculate ratio
responseIncrease = responseDuringFlooding.reset_index().merge(responseDuringNormal.reset_index(), on = 'DestinationID', how = 'left', suffixes = ('_flooding', '_normal'))
responseIncrease['Ratio'] = responseIncrease['ResponseTime_flooding'] / responseIncrease['ResponseTime_normal']
responseIncrease = responseIncrease[~responseIncrease.Ratio.isna()]
# add info to roads
roads_0 = roads.copy()
roads_0 = roads_0[roads_0.OBJECTID.isin(responseIncrease.DestinationID.values.tolist())]
roads_0 = roads_0.merge(responseIncrease[['DestinationID', 'Ratio']], how = 'left', left_on = 'OBJECTID', right_on = 'DestinationID')
# draw real response time increase ratio
# roads_0_select = roads_0[(roads_0.Ratio < 100) & (roads_0.Ratio > 4)]
# showRoadsWithValues(roads_0_select, 'Ratio', vmax = roads_0_select.Ratio.max(), vmin = 4, saveAddr = './fig/realResponse_increaseRatio_allPoints', figsize = (30, 30))

In [ ]:
# travel distance increase ratio estimation
roads_0 = roads.copy()
roads_0 = roads_0[(roads_0.travelTimeIncreaseRatio > 4)]
showRoadsWithValues(roads_0, 'travelTimeIncreaseRatio',
                    vmax = 10,
                    vmin = 4,
                    saveAddr = './fig/response_increaseRatio_max10',
                    figsize = (30,30))

In [ ]:
# get response time during normal time in real world dataset
dataDuringNormal = data.loc[(data.index < '2016-10-08 18:00:00') | (data.index > '2016-10-9 18:00:00')]
responseDuringNormal = dataDuringNormal[['DestinationID', 'ResponseTime']].groupby('DestinationID').mean()
# calculate prediction of response time
roads_0 = roads.copy()
roads_0 = roads_0.merge(responseDuringNormal, left_on = 'OBJECTID', right_on = 'DestinationID', how = 'left')
roads_0['responseTimePred'] = roads_0.travelTimeIncreaseRatio * roads_0.ResponseTime
roads_0 = roads_0[~roads_0.responseTimePred.isna()]
roads_0 = roads_0[['OBJECTID', 'responseTimePred']]

# fill the nan in dataDuringFlooding
dataDuringFlooding = data.loc[(data.index > '2016-10-08 18:00:00') & (data.index < '2016-10-9 18:00:00')]
dataDuringFlooding_na = dataDuringFlooding[dataDuringFlooding.ResponseTime.isna()]
dataDuringFlooding_na = dataDuringFlooding_na.reset_index()
dataDuringFlooding_na.CloseDateTime = pd.to_datetime(dataDuringFlooding_na.CloseDateTime)
dataDuringFlooding_na.ResponseTime = dataDuringFlooding_na.CloseDateTime - dataDuringFlooding_na.CallDateTime
dataDuringFlooding_na['ResponseTime'] = dataDuringFlooding_na.ResponseTime.dt.total_seconds()
dataDuringFlooding_na = dataDuringFlooding_na.set_index('CallDateTime')
dataDuringFlooding.update(dataDuringFlooding_na['ResponseTime'])
# get the response time during flooding in real world dataset
responseDuringFlooding = dataDuringFlooding[['DestinationID', 'ResponseTime']].groupby('DestinationID').mean()
# project it on roads
roads_1 = roads.copy()
roads_1 = roads_1.merge(responseDuringFlooding, left_on = 'OBJECTID', right_on = 'DestinationID', how = 'left')
roads_1 = roads_1[~roads_1.ResponseTime.isna()]

# combine and organize prediction and true
roads_combined = roads_1.merge(roads_0, left_on = 'OBJECTID', right_on = 'OBJECTID', how = 'left')
roads_combined = roads_combined[~roads_combined.responseTimePred.isna()]

showRoadsWithValues(roads_combined, 'responseTimePred',
                    vmax = 1200,
                    vmin = 600,
                    saveAddr = './fig/responseTime_estimation',
                    figsize = (30,30))

showRoadsWithValues(roads_combined, 'ResponseTime',
                    vmax = 1200,
                    vmin = 600,
                    saveAddr = './fig/responseTime_real',
                    figsize = (30,30))

# Model and training

## setup

In [ ]:
import torch
from torch import nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

from ray import tune
from ray.tune import CLIReporter
from ray.tune.schedulers import ASHAScheduler
from ray.tune.search.basic_variant import BasicVariantGenerator
import os

import tensorflow as tf
from tensorflow import keras

import plotly.express as px
import plotly.graph_objs as go
from plotly.subplots import make_subplots

import random
from darts.models import NaiveSeasonal
import matplotlib.pyplot as plt
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.stattools import ccf

In [ ]:
%pip install statsmodels
%pip install pmdarima

## preprocess data

In [ ]:
# select data and remove nan value
def dataCleaner(data, resStation = None, cutFloodingTime = True, timeIntervalShift = True):

    if cutFloodingTime == True:
        data = data.loc[:'2016-10-07']

    data_simpleLSTM = data[['EnRouteTime',
                            'DispatchTime',
                            'AssumedRouteLength',
                            'AverageSpeed',
                            'RescueSquadNumber',
                            ]]
    if resStation:
        data_simpleLSTM = data_simpleLSTM[data['RescueSquadNumber'] == resStation]

        ########## NOTE: time interval cal will be WRONG if no res station is selected
        data_simpleLSTM['TimeInterval'] = data_simpleLSTM.index.to_series().diff().dt.total_seconds()
        data_simpleLSTM = data_simpleLSTM.iloc[1:, :]
        if timeIntervalShift == True:
            data_simpleLSTM['TimeInterval+1'] = data_simpleLSTM['TimeInterval'].shift(-1)
            data_simpleLSTM = data_simpleLSTM.iloc[:-1].drop(['TimeInterval'], axis = 1)

    # replace route to inreachale place from nan to a high value
    routeLen_mean = data_simpleLSTM.AssumedRouteLength.mean()
    routeLen_std = data_simpleLSTM.AssumedRouteLength.std()
    data_simpleLSTM['AssumedRouteLength'] = data_simpleLSTM['AssumedRouteLength'].fillna(routeLen_mean + 3 * routeLen_std)

    # replace inf or nan speed with mean
    speedColumn = data_simpleLSTM.AverageSpeed
    speedColumn = speedColumn[~(speedColumn.isin([np.inf, np.nan]))]
    speed_mean = speedColumn.mean()
    speed_std = speedColumn.std()
    data_simpleLSTM['AverageSpeed'] = data_simpleLSTM['AverageSpeed'].replace(np.inf, speed_mean)
    data_simpleLSTM['AverageSpeed'] = data_simpleLSTM['AverageSpeed'].fillna(speed_mean)

    # fill en route time with mean
    data_simpleLSTM['EnRouteTime'] = data_simpleLSTM[['EnRouteTime']].fillna(data_simpleLSTM['EnRouteTime'].mean())

    # drop column
    data_simpleLSTM = data_simpleLSTM.drop('RescueSquadNumber', axis = 1)

    return data_simpleLSTM

def sequencesGeneration(data, endIndex, windowSize, startIndex = 0, stride = 1, downSample = 1):
    numSequence = (endIndex + 1) - (windowSize - 1)
    sequencesIndex = (
        startIndex + # start index of the whole data in use
        np.tile(np.arange(windowSize, step = downSample), (numSequence, 1)) + # window
        np.tile(np.arange(numSequence, step = stride), (windowSize, 1)).T # sequence
    )
    return data[sequencesIndex]

def splitData(sequences, valProportion, testProportion, xCount, yCount, shuffle = False):
    if shuffle == True:
        np.random.shuffle(sequences)

    valCount = int(np.floor(sequences.shape[0] * valProportion))
    testCount = int(np.floor(sequences.shape[0] * testProportion))
    trainCount = sequences.shape[0] - valCount - testCount

    trainX = sequences[:trainCount, :xCount, :]
    trainY = sequences[:trainCount, -yCount:, [0]]
    valX = sequences[trainCount: trainCount + valCount, :xCount, :]
    valY = sequences[trainCount: trainCount + valCount, -yCount:, [0]]
    testX = sequences[trainCount + valCount : trainCount + valCount + testCount, :xCount, :]
    testY = sequences[trainCount + valCount : trainCount + valCount + testCount, -yCount:, [0]]

    print('Dims: (sample, sequenceLen, feature)')
    print('trainX shape:', trainX.shape, ' trainY shape:', trainY.shape)
    print('valX shape:', valX.shape, ' valY shape:', valY.shape)
    print('testX shape:', testX.shape, ' testY shape:', testY.shape)
    return trainX, trainY, valX, valY, testX, testY


## vis

In [ ]:
# data_select = data_simpleLSTM.iloc[: 70]
# fig = go.Figure()
# fig.add_trace(go.Scatter(x = data_select.index, y = data_select['DispatchTime']))
# fig.add_trace(go.Scatter(x = data_select.index, y = data_select['EnRouteTime']))
# fig.show()

# data_select = data_simpleLSTM.iloc[: 200]
# fig = make_subplots(specs = [[{"secondary_y": True}]])
# fig.add_trace(go.Scatter(x = data_select.index, y = data_select['EnRouteTime']), secondary_y = False)
# fig.add_trace(go.Scatter(x = data_select.index, y = data_select['TimeInterval']), secondary_y = True)
# fig.show()

In [ ]:
# autocorelation or partical auto-cor plot of prep time
plot_pacf(dataCleaner(data, 'R14', cutFloodingTime = True)[['EnRouteTime']])
plt.show()

In [ ]:
# cross correlation plot
df = dataCleaner(data, 'R14', cutFloodingTime = True, timeIntervalShift = False)
cross_correlation = ccf(df['EnRouteTime'], df['TimeInterval'])
# plt.plot(cross_correlation)
cross_correlation[:10]

## LSTM

In [ ]:
# use the data of one station
sequenceLen = 15
yForward = 1
xCount = sequenceLen - yForward
data_simpleLSTM = dataCleaner(data, 'R14', cutFloodingTime = True)
sequences = sequencesGeneration(data_simpleLSTM.values, data_simpleLSTM.values.shape[0] - 1, 22)
trainX, trainY, valX, valY, testX, testY = splitData(sequences, 0.15, 0.15, xCount, yForward, shuffle = False)

In [ ]:
sequenceLen = 15
numFeature = 5
yForward = 1
xCount = sequenceLen - yForward
stationList = ['R14', 'R16', 'R08', 'R18', 'R10', 'R02', 'R09', 'R15', 'R21',
               'R04', 'R19', 'R05']
sequences = np.empty((1, sequenceLen, numFeature))
for station in stationList:
    data_simpleLSTM = dataCleaner(data, station, cutFloodingTime = True)

    data_simpleLSTM['EnRouteTime'] = data_simpleLSTM.EnRouteTime.clip(upper = data_simpleLSTM.EnRouteTime.mean() + 3 * data_simpleLSTM.EnRouteTime.std())
    # data_simpleLSTM['DispatchTime'] = data_simpleLSTM.DispatchTime.clip(upper = data_simpleLSTM.DispatchTime.mean() + 3 * data_simpleLSTM.DispatchTime.std())
    data_simpleLSTM['AssumedRouteLength'] = data_simpleLSTM.AssumedRouteLength.clip(upper = data_simpleLSTM.AssumedRouteLength.mean() + 3 * data_simpleLSTM.AssumedRouteLength.std())
    data_simpleLSTM['TimeInterval+1'] = data_simpleLSTM['TimeInterval+1'].clip(upper = data_simpleLSTM['TimeInterval+1'].mean() + 3 * data_simpleLSTM['TimeInterval+1'].std())

    sequence = sequencesGeneration(data_simpleLSTM.values, data_simpleLSTM.values.shape[0] - 1, sequenceLen)
    sequences = np.concatenate([sequences, sequence], axis = 0)
sequences = sequences[1:, :, :]

# addon = np.tile(sequences[:, -yForward:, -1:], (1, 1, numFeature)) ##### PENDING
# sequences_temp = np.concatenate((sequences[:, :-yForward, :], addon), axis = 1) ##### PENDING
# sequences = np.concatenate((sequences_temp, sequences[:, -yForward:, :]), axis = 1) ##### PENDING
# xCount += 1 ##### PENDING

trainX, trainY, valX, valY, testX, testY = splitData(sequences, 0.15, 0.15, xCount, yForward)

In [ ]:
def run_LSTM(lr, batch_size, yForward):
    model_LSTM = tf.keras.Sequential([
        keras.layers.BatchNormalization(),
        keras.layers.LSTM(128),
        keras.layers.Dense(yForward),
    ])
    model_LSTM.compile(optimizer = keras.optimizers.Adam(learning_rate = lr),
                loss = "mse")
    model_LSTM.fit(
        x = trainX,
        y = trainY,
        batch_size = batch_size,
        epochs = 500,
        validation_data = (valX, valY),
        callbacks = [
            keras.callbacks.EarlyStopping(
                monitor = "val_loss",
                min_delta = 0.001,
                patience = 3),
        ],
        verbose = 1,
    )
    prediction = model_LSTM.predict(testX)
    return prediction

prediction = run_LSTM(0.001, 512, yForward)

In [ ]:
class prepTimeDataset(Dataset):
    def __init__(self, X, Y):
        self.X = np.copy(X)
        self.Y = np.copy(Y)

    def __len__(self):
        return self.X.shape[0]

    def __getitem__(self, idx):
        feature = self.X[idx]
        target = self.Y[idx]
        return feature, target

class network(nn.Module):
    def __init__(self, numFeature, yForward):
        super().__init__()
        self.norm = nn.BatchNorm1d(numFeature)
        self.LSTM = nn.LSTM(numFeature, 512, num_layers = 1, batch_first = True)
        self.net1 = nn.Sequential(
            nn.ReLU(),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Linear(256, yForward),
        )

    def forward(self, x):
        # self.LSTM.flatten_parameters()
        x = torch.permute(x, (0, 2, 1))
        x = self.norm(x)
        x = torch.permute(x, (0, 2, 1))
        x, _ = self.LSTM(x)
        x = x[:, -1, :]
        out = self.net1(x)
        return out

class earlyStopper():
    def __init__(self, patience = 5, minDelta = 0.01):
        self.patience = patience
        self.minDelta = minDelta
        self.counter = 0
        self.minValLoss = np.inf

    def earlyStop(self, valLoss):
        if valLoss < (self.minValLoss - self.minDelta):
            self.minValLoss = valLoss
            self.counter = 0
        else:
            self.counter += 1
            if self.counter >= self.patience:
                return True
        return False

class prepTimePred():
    def __init__(self, numFeature, yForward):
        self.device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"
        print(f"Using {self.device} device")

        self.model = network(numFeature, yForward)
        self.model.to(self.device)
        print(self.model)

        self.ifTuning = False
        self.verbose = 1

    def _train(self, dataloader, model, optimizer, lossFunc):
        model.train()
        size = len(dataloader.dataset)
        for batch, (X, y) in enumerate(dataloader):
            X, y = X.to(self.device, dtype = torch.float), y.to(self.device, dtype = torch.float)
            pred = model(X)
            y = y[:, :, 0]
            loss = lossFunc(pred, y)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()

            if batch % 10 == 0:
                loss, current = loss.item(), (batch + 1) * len(X)
                if self.verbose == 1:
                    print(f"loss: {loss: >7f}  [{current: >5d} / {size: >5d}]")

    def _val(self, dataloader, model, lossFunc):
        model.eval()
        loss = 0
        with torch.no_grad():
            for X, y in dataloader:
                X, y = X.to(self.device, dtype = torch.float), y.to(self.device, dtype = torch.float)
                pred = model(X)
                y = y[:, :, 0]
                loss += lossFunc(pred, y).item()
        loss /= len(dataloader)
        if self.verbose == 1:
            print(f"Avg loss: {loss:>8f} \n")
        return loss

    def learn(self, config, trainX, trainY, valX, valY, epochs = 100):
        # load data
        trainDataloader = DataLoader(prepTimeDataset(trainX, trainY), batch_size = config["batchSize"])
        valDataloader = DataLoader(prepTimeDataset(valX, valY), batch_size = config["batchSize"])
        # optimzer and loss
        optimizer = torch.optim.Adam(self.model.parameters(), lr = config["lr"])
        lossFunc = nn.MSELoss()
        # learning
        earlyStop = earlyStopper(patience = 3, minDelta = 0)
        for t in range(epochs):
            if self.verbose == 1:
                print(f"Epoch {t+1}\n-------------------------------")
            self._train(trainDataloader, self.model, optimizer, lossFunc)
            valLoss = self._val(valDataloader, self.model, lossFunc)
            if self.ifTuning:
                tune.report(valLossFinalIter = valLoss)
            if earlyStop.earlyStop(valLoss):
                break
        if self.verbose == 1:
            print("Done!")

    def learn4tuning(self, trainX, trainY, valX, valY, numSamples, paramSpace):
        self.ifTuning = True
        self.verbose = 0
        trainable = tune.with_parameters(self.learn, trainX = trainX, trainY = trainY, valX = valX, valY = valY)
        trainableWithRes = tune.with_resources(trainable,
            resources = lambda x: {"gpu": 1} if self.device == 'cuda' else {"gpu": 0})
        scheduler = ASHAScheduler(
            metric = "valLossFinalIter",
            mode = "min",
            max_t = 500,
            reduction_factor = 2)
        tuner = tune.Tuner(
            trainableWithRes,
            param_space = paramSpace,
            tune_config = tune.TuneConfig(
                scheduler = scheduler,
                search_alg = BasicVariantGenerator(),
                num_samples = numSamples,
            )
        )
        results = tuner.fit()

        self.ifTuning = False
        tuningLog = {result.log_dir: result.metrics_dataframe for result in results}

    def predict(self, testX, testY):
        self.model.eval()
        testDataLoader = DataLoader(prepTimeDataset(testX, testY), batch_size = testX.shape[0])
        for batch, (X, y) in enumerate(testDataLoader):
            X = X.to(self.device, dtype = torch.float)
            with torch.no_grad():
                pred = self.model(X)
                if self.device == 'cuda':
                    return pred.cpu().detach().numpy()
                elif self.device == 'cpu':
                    return pred.detach().numpy()


prepTimePrediction = prepTimePred(numFeature, yForward)
prepTimePrediction.learn4tuning(trainX, trainY, valX, valY, 50,
 {"lr": tune.loguniform(1e-5, 1e-2),
  "batchSize": tune.choice([256, 512, 1024])
  })
# prepTimePrediction.learn({'lr': 0.0001, 'batchSize': 512}, trainX, trainY, valX, valY)
prediction = prepTimePrediction.predict(testX, testY)

In [ ]:
ranInt = random.randint(0, prediction.shape[0])
print(prediction[ranInt])
print(testY[ranInt][:,0])
print('mean of y', np.mean(testY))

## ARIMA

In [ ]:
from statsmodels.tsa.arima.model import ARIMA
from pmdarima import auto_arima

from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import r2_score

In [ ]:
data_ARIMA = dataCleaner(data, 'R14', cutFloodingTime = True)[['EnRouteTime']]
data_ARIMA

In [ ]:
model_auto = auto_arima(data_ARIMA , trace = True, error_action = 'ignore', suppress_warnings = True)
model_auto.fit(data_ARIMA)

In [ ]:
model = ARIMA(data_ARIMA, order=(3,2,2))
model_fit = model.fit()
model_fit.predict(start = len(data_ARIMA), end=len(data_ARIMA)+1)
mean_squared_error(data_ARIMA.values, model_fit.predict(start = 0, end=4025).values[1:])

# Experiments

In [ ]:
plt.style.use('seaborn')
fig, ax = plt.subplots(figsize=(30, 7.5))
t = range(len(dataTest_y_Flood))
ax.plot(t, predictions, label = 'prediction')
ax.plot(t, dataTest_y_Flood, label = 'ground truth')
ax.set_xlabel('Incident', fontsize = 20)
ax.set_ylabel('If accessible', fontsize = 20)
ax.legend(fontsize = 20)
ax.tick_params(axis='both', labelsize = 15)
#ax.set_title('Prediction vs Ground truth', fontsize = 25)

In [ ]:
from sklearn.metrics import confusion_matrix , classification_report
print(classification_report(dataTest_y_Flood, predictions))

In [ ]:
def calculateWaste (row):
    if row['If accessible Real'] == 1 and row['If accessible Predicted'] == 0:
        return 1
    else:
        return 0

def calculateUnknownDanger (row):
    if row['If accessible Real'] == 0 and row['If accessible Predicted'] == 1:
        return 1
    else:
        return 0

dataFlood = dataProcessed.loc['2016-10-09' : '2016-10-09']
dataFlood['If accessible Real'] = dataFlood['Accessibility'].astype('int64')
dataFlood['If accessible Predicted'] = [element[0] for element in predictions]
dataFlood['Waste'] = dataFlood.apply(calculateWaste, axis = 1)
dataFlood['Unknown Danger'] = dataFlood.apply(calculateUnknownDanger, axis = 1)
dataFlood['Error Type'] = (dataFlood['Waste'] + dataFlood['Unknown Danger'] * 2).astype('string') # 1 mean wastes, 2 means potential danger
dataFlood['Error Type'] = dataFlood['Error Type'].replace('1', 'Type 1').replace('2', 'Type 2')
pd.set_option('display.max_rows', 20)
display(dataFlood)

# visualization
px.set_mapbox_access_token(open("mapboxToken.txt").read())
fig1 = px.scatter_mapbox(dataFlood.loc[lambda df: df['Error Type'] != '0'],
                        lat="latitude", lon="longitude",
                        color = "Error Type", #size = "Response Time",
                        size_max = 15, zoom = 10, width = 575, height = 500)
fig1.show()

fig2 = px.scatter_mapbox(dataFlood.loc[lambda df: (df['Error Type'] != '0') &
                                      (df['Error Type'] != '1')],
                        lat="latitude", lon="longitude",
                        color = "Error Type", size = "Response Time",
                        size_max = 30, zoom = 10, width = 550, height = 500)
fig2.update_layout(showlegend=False)
fig2.show()

In [ ]:
# import
import geopandas as gpd
from shapely.geometry import Point
shp = gpd.read_file('./data/VB_City_Boundary.geojson')
shp.crs = 'CRS84'

# generate all points
numOfPointsOneDimX = 50
deltaX = shp.bounds.maxx - shp.bounds.minx
deltaY = shp.bounds.maxy - shp.bounds.miny
numOfPointsOneDimY = numOfPointsOneDimX * (deltaY / deltaX)

xCorList = np.arange(float(shp.bounds.minx), float(shp.bounds.maxx), float(deltaX / numOfPointsOneDimX))
yCorList = np.arange(float(shp.bounds.miny), float(shp.bounds.maxy), float(deltaY / numOfPointsOneDimY))
xyPointList = [Point(x, y) for x in xCorList for y in yCorList]

# select points within the city
samplePoints = gpd.GeoSeries(xyPointList)
samplePoints.crs = 'CRS84'
withinOrNot = samplePoints.within(shp['geometry'].values[0])
gdf = pd.concat([samplePoints, withinOrNot], axis = 1)
gdf.crs = 'CRS84'
gdfSelected = gdf.loc[gdf[1] == True]
display(gdfSelected)

In [ ]:
# do the prediction for these points
gdfSelected['latitude'] = gdfSelected[0].values.y
gdfSelected['longitude'] = gdfSelected[0].values.x

def addTimeFeature(gdf, hourInDay, dayOfWeek):
    gdf_out = gdf.copy()
    gdf_out['Hour in Day'] = hourInDay
    gdf_out['Day of Week'] = dayOfWeek
    return gdf_out

gdfSelected_withTime = addTimeFeature(gdfSelected, 0, 6) #The day of week is 1, because it is the normalized value, flooding day is Sunday
for hour in range(23):
    gdfSelected_withTime = pd.concat([gdfSelected_withTime, addTimeFeature(gdfSelected, hour + 1, 6)])

gdfForPrediction = gdfSelected_withTime.reset_index().loc[:,['latitude', 'longitude', 'Hour in Day']]
gdfForPrediction_norm = normalizer(gdfForPrediction.copy())
gdfForPrediction_norm['Day of Week'] = 1 #The day of week is 1, because it is the normalized value, flooding day is Sunday
gdfForPrediction_norm

In [ ]:
predictionsFull = model.predict(gdfForPrediction_norm.values)
predictionsFull = np.where(predictionsFull < 0.5, 0, 1).tolist()
print(len(predictionsFull))

In [ ]:
dataFloodFull = gdfForPrediction.copy()
dataFloodFull['If accessible'] = [element[0] for element in predictionsFull]
dataFloodFull['If accessible'] = dataFloodFull['If accessible'].astype('string').replace('1', 'Accessible').replace('0', 'Inaccessible')
display(dataFloodFull)

# visualization
px.set_mapbox_access_token(open("mapboxToken.txt").read())
fig = px.scatter_mapbox(dataFloodFull.loc[dataFloodFull['Hour in Day'] == 23],
                        lat = "latitude", lon = "longitude",
                        color = "If accessible", #size = "Response Time",
                        zoom = 9.5, opacity = 0.7, width = 600, height =700)
fig.show()